### Importación de Librerías

In [5]:
import simpy
import pandas as pd
import random

### Cabina de Venta

In [7]:
class TicketBooth:
    def __init__(self, env, name, service_time, capacity):
        self.env = env
        self.name = name
        self.service_time = service_time
        self.resource = simpy.Resource(env, capacity=capacity)

        # DataFrames para seguimiento
        self.dfBooth = pd.DataFrame({
            "name": [name],
            "service_time": [service_time]
        })
        self.dfEvents = pd.DataFrame()

    def sell_ticket(self, customer_name):
        yield self.env.timeout(random.uniform(20, 60))  # Tiempo de servicio aleatorio
        print(f"{self.env.now}: {self.name} vendió un boleto a {customer_name} en {self.service_time} segundos.")
        df = pd.DataFrame({
            "time": [self.env.now],
            "booth": [self.name],
            "customer": [customer_name],
            "event": ["ticket sold"]
        })
        self.dfEvents = pd.concat([self.dfEvents, df])

### Cliente

In [9]:
class Customer:
    def __init__(self, env, name, ticket_booth):
        self.env = env
        self.name = name
        self.ticket_booth = ticket_booth
        env.process(self.wait_in_line())

        # DataFrame para seguimiento
        self.dfCustomer = pd.DataFrame({
            "name": [name],
            "ticket_booth": [ticket_booth.name]
        })
        self.dfEvents = pd.DataFrame()

    def wait_in_line(self):
        print(f"{self.env.now}: {self.name} está esperando en {self.ticket_booth.name}.")
        df = pd.DataFrame({
            "time": [self.env.now],
            "booth": [self.ticket_booth.name],
            "customer": [self.name],
            "event": ["waiting in line"]
        })
        self.dfEvents = pd.concat([self.dfEvents, df])
        
        with self.ticket_booth.resource.request() as request:
            yield request
            print(f"{self.env.now}: {self.name} está listo para ser atendido en {self.ticket_booth.name}.")
            yield self.env.process(self.ticket_booth.sell_ticket(self.name))
            print(f"{self.env.now}: {self.name} fue atendido en {self.ticket_booth.name} y compró su boleto.")
            df = pd.DataFrame({
                "time": [self.env.now],
                "booth": [self.ticket_booth.name],
                "customer": [self.name],
                "event": ["finished"]
            })
            self.dfEvents = pd.concat([self.dfEvents, df])
            yield self.env.timeout(1)

### Setup

In [11]:
class Setup:
    def __init__(self, env, num_booths, num_customers):
        self.env = env
        self.customers = []
        self.num_customers = num_customers
        self.booths = [TicketBooth(
            env,
            f"Booth {i}",
            random.randint(20, 60),  # Tiempo de servicio aleatorio entre 20-60 segundos
            capacity=1  # Cada cabina puede atender a un cliente a la vez
        ) for i in range(num_booths)]

        env.process(self.customer_arrival())

    def customer_arrival(self):
        for i in range(self.num_customers):
            self.customers.append(Customer(
                env,
                f"Customer {i}",
                random.choice(self.booths)
            ))
            yield self.env.timeout(random.expovariate(1/50))  # Distribución exponencial para llegadas


### Simulación 

In [13]:
random.seed(42)
env = simpy.Environment()

setup = Setup(env, num_booths=3, num_customers=50)

env.run(until=2000)  # Simulación por 2000 segundos

0: Customer 0 está esperando en Booth 2.
0: Customer 0 está listo para ser atendido en Booth 2.
16.081203203748277: Customer 1 está esperando en Booth 2.
21.488042398492254: Customer 2 está esperando en Booth 2.
28.92842952595291: Booth 2 vendió un boleto a Customer 0 en 21 segundos.
28.92842952595291: Customer 0 fue atendido en Booth 2 y compró su boleto.
29.92842952595291: Customer 1 está listo para ser atendido en Booth 2.
53.405982831129556: Booth 2 vendió un boleto a Customer 1 en 21 segundos.
53.405982831129556: Customer 1 fue atendido en Booth 2 y compró su boleto.
54.405982831129556: Customer 2 está listo para ser atendido en Booth 2.
91.28285561854037: Booth 2 vendió un boleto a Customer 2 en 21 segundos.
91.28285561854037: Customer 2 fue atendido en Booth 2 y compró su boleto.
132.85244737606126: Customer 3 está esperando en Booth 0.
132.85244737606126: Customer 3 está listo para ser atendido en Booth 0.
137.77142985898408: Customer 4 está esperando en Booth 2.
137.7714298589

### Recolectamos los datos

In [15]:
df_events = pd.DataFrame()
df_booths = pd.DataFrame()
df_customers = pd.DataFrame()

for booth in setup.booths:
    df_booths = pd.concat([df_booths, booth.dfBooth])
    df_events = pd.concat([df_events, booth.dfEvents])

for customer in setup.customers:
    df_customers = pd.concat([df_customers, customer.dfCustomer])
    df_events = pd.concat([df_events, customer.dfEvents])

### Análisis de la simulación

In [55]:
# Cuántos boletos vendió cada cabina
summBooths = df_customers.groupby(
    ["ticket_booth"], as_index=False
).agg({
    "name": ["count"]
})
summBooths.columns = ["ticket_booth", "num_customers"]

df_summary = df_booths.merge(
    summBooths,
    left_on="name",
    right_on="ticket_booth",
    how="inner"
)


In [53]:
df_summary

,name,service_time,ticket_booth,num_customers
0,Booth 0,60,Booth 0,11
1,Booth 1,27,Booth 1,19
2,Booth 2,21,Booth 2,20


In [18]:
# Análisis de eventos por cliente
sample2 = df_events.sort_values(["customer", "time"])
sample2['lead'] = sample2.groupby('customer')['time'].shift(-1)
sample2["time_on_event"] = sample2["lead"] - sample2["time"]

In [51]:
# Clientes que esperaron más de 1 segundo para ser atendidos
person_waiting = sample2.loc[
    (sample2['time_on_event'] > 10) &
    (sample2['event'] == "waiting in line"), :
]

print(f"Total de clientes que esperaron más de 10 segundo: {person_waiting.shape[0]}")


Total de clientes que esperaron más de 10 segundo: 48


In [59]:
# Tiempo promedio de espera por cabina
avg_waiting_time = person_waiting.groupby(
    "booth", as_index=False
).agg({
    "time_on_event": ["mean"],
    "customer": ["count"]
})
avg_waiting_time.columns = ["booth", "avg_wait_time", "num_customers"]

In [57]:
avg_waiting_time

,booth,avg_wait_time,num_customers
0,Booth 0,32.971832,11
1,Booth 1,82.699063,17
2,Booth 2,61.298278,20


In [61]:
# Tiempo máximo que las cabinas estuvieron activas
timemax = df_events.groupby(
    "booth", as_index=False
).agg({
    "time": ["max"]
})

timemax.columns = ["booth", "time"]
timemax["time"] = timemax["time"] / 60 / 60  # Convertir a horas

In [63]:
timemax

,booth,time
0,Booth 0,0.476331
1,Booth 1,0.542499
2,Booth 2,0.519616


In [68]:
# Cuánto tiempo estuvieron activas vs. paradas
printing = sample2.loc[
    sample2["event"] == "ticket sold",
    :
].groupby(
    ["booth"], as_index=False
).agg({
    "time_on_event": ["sum"]
})

printing.columns = ["booth", "time_used"]
printing["time_used"] = printing["time_used"] / 60 / 60  # Convertir a horas
printing["time_slept"] = 2000 / 60 / 60 - printing["time_used"]  # Tiempo total de simulación - tiempo usado

In [66]:
printing

,booth,time_used,time_slept
0,Booth 0,0.0,0.555556
1,Booth 1,0.0,0.555556
2,Booth 2,0.0,0.555556


In [72]:
# Tiempo promedio de inactividad entre clientes
sample3 = df_events.sort_values(["booth", "time"])
sample3["lead"] = sample3.groupby("booth")['time'].shift(-1)
sample3["time_on_event"] = sample3["lead"] - sample3["time"]

print_waiting = sample3.loc[
    sample3["event"] == "finished", :
].groupby(
    ["booth"], as_index=False
).agg({
    "time_on_event": ["mean", "median"]
})

print_waiting.columns = ["booth", "mean_wait", "median_wait"]
print_waiting["mean_wait"] = print_waiting["mean_wait"] / 60  # Convertir a minutos
print_waiting["median_wait"] = print_waiting["median_wait"] / 60  # Convertir a minutos

In [70]:
print_waiting

,booth,mean_wait,median_wait
0,Booth 0,2.032080,1.287126
1,Booth 1,1.202135,0.819715
2,Booth 2,1.227737,0.629680
